In [1]:
!pip install -q huggingface_hub
!pip install litellm


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from dotenv import load_dotenv
import os

load_dotenv()
HF_Token = os.getenv("HF_TOKEN")


In [2]:
from smolagents import LiteLLMModel
import os
from huggingface_hub import InferenceClient
model = LiteLLMModel(
    model_id="ollama_chat/qwen2.5:latest",
    api_base="http://localhost:11434",
    num_ctx=8192,
)
client = InferenceClient(provider="hf-inference", model="meta-llama/Llama-3.3-70B-Instruct");

In [4]:
# This system prompt is a bit more complex and actually contains the function description already appended.
# Here we suppose that the textual description of the tools have already been appended
SYSTEM_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

get_weather: Get the current weather in a given location

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are:
get_weather: Get the current weather in a given location, args: {{"location": {{"type": "string"}}}}
example use :
```
{{
  "action": "get_weather",
  "action_input": {"location": "New York"}
}}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about one action to take. Only one action at a time in this format:
Action:
```
$JSON_BLOB
```
Observation: the result of the action. This Observation is unique, complete, and the source of truth.
... (this Thought/Action/Observation can repeat N times, you should take several steps when needed. The $JSON_BLOB must be formatted as markdown and only use a SINGLE action at a time.)

You must always end your output with the following format:

Thought: I now know the final answer
Final Answer: the final answer to the original input question

Now begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer. """

In [8]:
# Since we are running the "text_generation", we need to add the right special tokens.
prompt=f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{SYSTEM_PROMPT}
<|eot_id|><|start_header_id|>user<|end_header_id|>
What's the weather in London ?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [9]:
# The answer was hallucinated by the model. We need to stop to actually execute the function!
output = client.text_generation(
    prompt,
    max_new_tokens=150,
    stop=["Observation:"] # Let's stop before any actual function is called
)

print(output)

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/meta-llama/Llama-3.3-70B-Instruct (Request ID: Root=1-68413160-2ae3c5c10eb8d5df3ff122cd;89bd95a0-a279-4377-8493-d687958b723b)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [ ]:
# Dummy function
def get_weather(location):
    return f"the weather in {location} is sunny with low temperatures. \n"

get_weather('London')

In [10]:
# Let's concatenate the base prompt, the completion until function execution and the result of the function as an Observation
new_prompt=prompt+output+get_weather('London')
print(new_prompt)

NameError: name 'output' is not defined

In [ ]:
final_output = client.text_generation(
    new_prompt,
    max_new_tokens=200,
)

print(final_output)

In [ ]:
# Tool calling agents
for query in [
    "Best catering services in Gotham City", 
    "Party theme ideas for superheroes"
]:
    print(web_search(f"Search for: {query}"))
    
    

In [3]:
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool

agent = ToolCallingAgent(tools=[DuckDuckGoSearchTool()], model=model)

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5:latest ─────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': "best music recommendations for a party at Wayne's         │
│ mansion"}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: ## Search Results

|The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
The best party songs 1. "September" - Earth, Wind & Fire (1978) Quite possibly the best party song. An infectious 
mix of funk and soul, "September" is celebrated for its upbeat melody and "ba-dee-ya" chorus, making it a timeless 
dance favorite.

|late nights in the wayne manor part 2 | a bruce wayne rock/blues ...](https://www.youtube.com/watch?v=DBRSHk2XpmU)
these are songs that bruce wayne listens to while wandering the halls of the great wayne manor. this is part 2.find
this playlist on my spotify: https://open...

|Similar Songs Finder | Spotify Playlist Generator - Chosic](https://www.chosic.com/playlist-generator/)
Need to find similar songs to a song you like? To music you love? A playlist created for you in seconds! With 
Spotify Playlist Generator online tool, create awesome playlists in seconds, featuring similar songs to those you 
love. Whether by songs, artists, genres, moods, or based on a playlist, just start with what you have in mind, and 
it will provide you with plenty of song recommendations ...

|50 Songs on Every Event Planner's Playlist - 
Eventbrite](https://www.eventbrite.com/blog/event-planning-playlist-ds00/)
Music sets the mood and provides the soundtrack (literally) for a memorable and exciting time. While the right 
songs can enhance the experience, the wrong event music can throw off the vibe. For example, fast-paced songs 
probably aren't the best fit for a formal gala. And smooth jazz is likely to lull your guests at a motivational 
conference.

|Party Music Guide: Ultimate Tips for the Perfect 
Playlist](https://thebackstage-deezer.com/music/perfect-party-music-playlist/)
Check out the best party playlists and top party songs to ensure your next party is packed! The most popular party 
songs are here, just hit play. ... to decor. But, most of all, you need to have fantastic music. We recommend you 
get at least three hours' worth of party music queued and ready — that's about 75 songs. Lucky for you, we've ...

|40 Best Party Songs | Songs To Dance To, Ranked By Our Editors - Time 
Out](https://www.timeout.com/music/best-party-songs)
The 40 best party songs ever made. It's party time guys - don your dancing shoes and have your speaker at the 
ready. Monday 12 May 2025. Share. Copy Link. Facebook Twitter Pinterest Email ...

|Song Lists to help you plan the perfect music for your event!](http://pianistforparties.com/songs/)
Song ideas to help you plan music for your wedding, corporate event or memorial service, by Event Pianist, Sharon 
Planer. ... Reception & Party. 100 Years. A Day in the Life of a Fool. Accidentally in Love. All Blues. Always. 
Ain't Misbehaving. ... Mansion Over the Hilltop. My Heart is Longing. My Way. Near to the Heart of God. Old Rugged 
Cross ...

|200 Classic House Party Songs Everyone Knows | The Best ... - 
iSpyTunes](https://www.ispytunes.com/post/house-party-songs)
Next, let's get into reviewing 20 of the best songs for your next house party. 1. rockstar by Post Malone (feat. 21
Savage) "rockstar" by Post Malone (feat. 21 Savage) is an incredibly popular song with a great instrumental and an 
infectious chorus, which makes it a perfect song for playing at parties. The whole song is like a giant energy ...

|Top Playlists; Find Your Perfect Music Mix | 
iHeart](https://www.iheart.com/playlist/collections/perfect-for/activities/party/)
Discover and stream the best playlists, handcrafted by the music experts at iHeart. Discover and stream the best 
playlists, handcrafted by the music experts at iHeart. For You; Your Library; Live Radio; Podcasts; ... 2010s Party
Hits. Sunday Brunch. Big Game Party. Tulum Party. Jazz Party. Feelin' Good. Country Party. Vocal House Party. 10s 
...

|The Top 100 Best Party Songs of All Time - YouTube 
Music](https://music.youtube.com/playlist?list=PL10BZg10BCDQxKXg0qN_-Y6RwvfdDvJdy)


[Step 1: Duration 21.03 seconds| Input tokens: 1,107 | Output tokens: 29]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': "Based on the search results, some of the best party    │
│ songs for a Wayne's mansion party include 'September' by Earth, Wind & Fire, 'rockstar' by Post Malone (feat.   │
│ 21 Savage), and 'The Top 100 Best Party Songs of All Time' playlist from YouTube Music. Additionally, you can   │
│ explore the 'Best Party Songs That Will Get Everyone Dancing' list from Gear4music for more options."}          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Based on the search results, some of the best party songs for a Wayne's mansion party include 
'September' by Earth, Wind & Fire, 'rockstar' by Post Malone (feat. 21 Savage), and 'The Top 100 Best Party Songs 
of All Time' playlist from YouTube Music. Additionally, you can explore the 'Best Party Songs That Will Get 
Everyone Dancing' list from Gear4music for more options.

[Step 2: Duration 20.82 seconds| Input tokens: 3,367 | Output tokens: 137]

"Based on the search results, some of the best party songs for a Wayne's mansion party include 'September' by Earth, Wind & Fire, 'rockstar' by Post Malone (feat. 21 Savage), and 'The Top 100 Best Party Songs of All Time' playlist from YouTube Music. Additionally, you can explore the 'Best Party Songs That Will Get Everyone Dancing' list from Gear4music for more options."